In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [25]:
import os
import sys
from datetime import datetime
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import cvxopt
import random
    
aeq_dir = "/mnt/c/Users/jan.zill/code/aequilibrae"
sys.path.append(aeq_dir)
from aequilibrae.paths.path_based.cythoncodetest import OpenBenchmark
# import node
# import link
# #import quadraticassignmentcyt
from aequilibrae.paths.path_based import TrafficAssignmentCy
aequ_dir = "/mnt/c/Users/jan.zill/code/aequilibrae"
sys.path.append(aequ_dir)

from aequilibrae.paths import TrafficAssignment  # noqa: E402
from aequilibrae.paths import Graph  # noqa: E402
from aequilibrae.paths.traffic_class import TrafficClass  # noqa: E402
from aequilibrae.matrix import AequilibraeMatrix, AequilibraeData  # noqa: E402
import matplotlib.pyplot as plt  # noqa: E402

from aequilibrae import logger  # noqa: E402
import logging  # noqa: E402

stdout_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stdout_handler)

In [4]:
code_dir = "/mnt/c/Users/jan.zill/code"
tntp_dir = os.path.join(code_dir, "TransportationNetworks_aequilibrae")
scenario = "Anaheim"
folder = os.path.join(tntp_dir, scenario)
link_file =  "Anaheim_net.tntp"
trip_file =  "Anaheim_trips.tntp"

block_centroids = False

In [14]:
net = pd.read_csv(os.path.join(tntp_dir, scenario, link_file), skiprows=7, sep="\t")
net = net.reset_index().rename(columns={"index": "link_id"})

if scenario == "Anaheim":
    cols_ = ["link_id", "Tail", "Head", "Free Flow Time (min)", "Capacity (veh/h)", "B", "Power"]
    col_names = ["link_id", "a_node", "b_node", "time", "capacity", "alpha", "beta"]
elif scenario == "Berlin-Center":
    cols_ = ["link_id", "Init node ", "Term node ", "Free Flow Time ", "Capacity ", "B ", "Power "]
    col_names = ["link_id", "a_node", "b_node", "time", "capacity", "alpha", "beta"]
else:
    raise ValueError(f"Scenario {scenario} needs network column name check")

network = net[cols_]
network.columns = col_names
network = network.assign(direction=1)

g = Graph()
g.cost = network["time"].values
g.capacity = network["capacity"].values
g.free_flow_time = network["time"].values

g.network = network.to_records(index=False)
g.network_ok = True
g.status = "OK"

# # Loads and prepares the matrix. NOTE: This assumes conversion to aem has been done, Pedro should have a gist online. I also would have that code somewhere.
mat = AequilibraeMatrix()
mat.load(os.path.join(tntp_dir, scenario, "demand.aem"))
mat.computational_view(["matrix"])
zones = mat.zones
index = np.arange(zones) + 1

g.prepare_graph(index)
g.set_graph("time")
g.cost = np.array(g.cost, copy=True)
g.set_skimming(["time"])
# this might lead to large diffs:
g.set_blocked_centroid_flows(block_centroids)

# # Creates the assignment class
assigclass = TrafficClass(g, mat)

# Instantiates the traffic assignment problem
assig = TrafficAssignment()
assig.set_classes(assigclass)

# configures it properly
assig.set_vdf("BPR")
#if scenario == "Anaheim":
#    assig.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
#elif scenario == "Berlin-Center":
assig.set_vdf_parameters({"alpha": "alpha", "beta": "beta"})
#else:
#    raise ValueError(f"Scenario {scenario} needs BPR params")

assig.set_capacity_field("capacity")
assig.set_time_field("time")
# could be assig.set_algorithm('frank-wolfe')
#assig.set_algorithm("bfw")


# # Execute the assignment
# assig.execute()
# # the results are within each traffic class only one, in this case
# # assigclass.results.link_loads
# solution = pd.DataFrame(assigclass.graph.network)
# solution["flow"] = assigclass.results.link_loads


In [6]:
net_df = pd.DataFrame(assig.classes[0].graph.network)

In [7]:
net_df.head(3)

,link_id,a_node,b_node,time,capacity,alpha,beta,direction
0,0,1,117,1.090458,9000,0.15,4,1
1,1,2,87,1.090458,9000,0.15,4,1
2,2,3,74,1.090458,9000,0.15,4,1


In [8]:
#if isinstance(self.vdf_parameters[p1], str):
#    array = np.array(self.classes[0].graph.graph[p], copy=True).astype(np.float64)

In [9]:
from aequilibrae.paths.path_based.node import Node
from aequilibrae.paths.path_based.link import Link

In [10]:
graph_ = pd.DataFrame(assig.classes[0].graph.graph)

In [26]:
assig.set_algorithm("bush-based")

assig.execute()

bush-based Assignment STATS
Iteration, RelativeGap, stepsize
 Initialised data structures, num nodes = 416, num links = 914
 0th iteration done, cost = 1264524.25


In [21]:
#assig.assignment.initialise_data_structures()